# Runoff Mapping Example
This notebook demonstrates how to use the mapping module of mom6_bathy to create an ESMF mapping file from a runoff source grid (JRA025) to a target MOM6 grid (tx2_3v2).

In [ ]:
import xarray as xr
from scipy.sparse import coo_matrix
from mom6_bathy.mapping import generate_ESMF_map_via_esmpy, generate_ESMF_map, compute_smoothing_weights

## Step 1/3: Generate a nearest neighbor mapping (ROF -> OCN)

In [2]:
# Mesh files (input)
rof_mesh_path = '/glade/p/cesmdata/cseg/inputdata/share/meshes/JRA025m.170209_ESMFmesh.nc'
ocn_mesh_path = '/glade/p/cesmdata/cseg/inputdata/share/meshes/tx2_3v2_230415_ESMFmesh.nc'
rof_mesh = xr.open_dataset(rof_mesh_path)
ocn_mesh = xr.open_dataset(ocn_mesh_path)

In [3]:
# Nearest neighbor map (intermediate output)
nn_map_path = '/glade/derecho/scratch/altuntas/croc_map_JRA025m_to_t232.nc'

# Smoothed nearest neighbor mapping file (final output)
rmax = 250.0  # Maximum distance for smoothing weights
fold = 250.0  # Folding factor for smoothing weights
nnsm_map_path = f"/glade/derecho/scratch/altuntas/croc_map_JRA025m_to_e{int(fold)}_r{int(rmax)}.nc"

In [4]:
# Final mapping file (to be generated)
nnsm_map_path

'/glade/derecho/scratch/altuntas/croc_map_JRA025m_to_e250_r250.nc'

In [5]:
generate_ESMF_map_via_esmpy(
    src_mesh_path=rof_mesh_path,
    dst_mesh_path=ocn_mesh_path,
    mapping_file=nn_map_path,
    method='nearest_d2s',
    area_normalization=True,
)

In [6]:
nn_map = xr.open_dataset(nn_map_path)

## Step 2/3: Compute smoothing weights (OCN -> OCN)

In [7]:
%%time
sw = compute_smoothing_weights(
    mesh_ds=ocn_mesh,
    rmax=rmax,  # Maximum distance for smoothing weights
    fold=fold,  # Folding factor for smoothing weights
)

CPU times: user 18.6 s, sys: 1.84 s, total: 20.4 s
Wall time: 20.4 s


## Step 3/3: Apply smoothing to (ROF -> OCN) map


In [8]:
src_nx = len(nn_map.ni_a)
src_ny = len(nn_map.nj_a)
dst_nx = len(nn_map.ni_b)
dst_ny = len(nn_map.nj_b)

In [9]:
# Create a COO matrix of the mapping weights (generated via esmpy)
S_coo = coo_matrix((nn_map.S.data, (nn_map.row.data-1, nn_map.col.data-1)), shape=(dst_nx*dst_ny, src_nx*src_ny))

In [10]:
# Apply smoothing by multiplying (transpose of) S_coo and smoothing weights (and taking transpose again):
S_smooth = S_coo.transpose().dot(sw).transpose()

In [11]:
type(S_smooth)

scipy.sparse._csc.csc_matrix

In [12]:
# Transform from CSC to COO:
S_smooth = S_smooth.tocoo()

In [13]:
# Write new mapping file:
generate_ESMF_map(
    src_mesh=rof_mesh_path,
    dst_mesh=ocn_mesh_path,
    filename=nnsm_map_path,
    weights_coo = S_smooth,
    area_normalization=False, # already applied to non-smooth map 
)